In [7]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import math

matplotlib.use('Qt5Agg')

DATA_PATH = "C:\\Users\\gaoch\\MA\\Badminton weights\\P01_B\\"

SHEET_NAME = ["Segment Position"]

SIMPLIFIED_JOINTS = ["Head", "Neck", "Left Upper Arm", "Right Upper Arm", "Left Forearm", "Right Forearm",
                     "Left Hand", "Right Hand", "Pelvis", "Left Upper Leg", "Right Upper Leg",
                     "Left Lower Leg", "Right Lower Leg", "Left Foot", "Right Foot"]

SKELETON_CONNECTION_MAP = [["Head", "Neck"],
                         ["Neck", "Left Upper Arm"],
                         ["Left Upper Arm", "Left Forearm"],
                         ["Left Forearm", "Left Hand"],
                         ["Neck", "Right Upper Arm"],
                         ["Right Upper Arm", "Right Forearm"],
                         ["Right Forearm", "Right Hand"],
                         ["Neck", "Pelvis"],
                         ["Pelvis", "Left Upper Leg"],
                         ["Left Upper Leg", "Left Lower Leg"],
                         ["Left Lower Leg", "Left Foot"],
                         ["Pelvis", "Right Upper Leg"],
                         ["Right Upper Leg", "Right Lower Leg"],
                         ["Right Lower Leg", "Right Foot"]]

used_cols = []
for joints in SIMPLIFIED_JOINTS:
    used_cols.append(joints+" x")
    used_cols.append(joints+" y")
    used_cols.append(joints+" z")

def read_data(file_name):
    return pd.read_excel(DATA_PATH + file_name,
                         sheet_name = SHEET_NAME,
                         usecols = used_cols)


motion_recording = read_data("P01_B_noweightwithgloves.xlsx")


In [61]:
def update(index, recordings, lines):
    chosen_frame =  recordings.iloc[index]
    for line, joint_connection in zip(lines, SKELETON_CONNECTION_MAP):
        endpoint_0 = joint_connection[0]
        endpoint_1 = joint_connection[1]

        line_x = [chosen_frame[endpoint_0 + " x"], chosen_frame[endpoint_1 + " x"]]
        line_y = [chosen_frame[endpoint_0 + " y"], chosen_frame[endpoint_1 + " y"]]
        line_z = [chosen_frame[endpoint_0 + " z"], chosen_frame[endpoint_1 + " z"]]
        line.set_data([line_x, line_y])
        line.set_3d_properties(line_z)

def centre(recording):
    columns_with_x = recording.columns[recording.columns.str.contains("x")]
    columns_with_y = recording.columns[recording.columns.str.contains("y")]
    columns_with_z = recording.columns[recording.columns.str.contains("z")]
    recording[columns_with_x] = recording[columns_with_x].sub(recording["Pelvis x"],axis='index')
    recording[columns_with_y] = recording[columns_with_y].sub(recording["Pelvis y"],axis='index')
    recording[columns_with_z] = recording[columns_with_z].sub(recording["Pelvis z"],axis='index')
    return recording

def get_sin_cos(opposite, adjacent):
    hypotenuse = math.sqrt(adjacent ** 2 + opposite ** 2)
    return opposite/hypotenuse, adjacent/hypotenuse

def rotate(frame, sin, cos):
    print(frame.name)
    rotation_matrix = np.array([[cos, sin, 0],[-sin, cos, 0],[0, 0, 1]])
    result_array = []
    for joint in range(0, len(frame), 3):
        joint_vector = np.array([[frame.iloc[joint]], [frame.iloc[joint+1]], [frame.iloc[joint+2]]])
        result_array = np.append(result_array, np.dot(rotation_matrix, joint_vector).T)
    return pd.Series(result_array)

def confront(recording):
    sin_cos = pd.DataFrame()
    sin_cos[["sin","cos"]] = recording.apply(lambda x: get_sin_cos(x["Left Upper Arm y"] - x["Right Upper Arm y"], x["Left Upper Arm x"] - x["Right Upper Arm x"]), axis=1, result_type="expand")
    recording[recording.columns] = recording.apply(lambda x: rotate(x, sin_cos["sin"][x.name], sin_cos["cos"][x.name]), axis=1, result_type="expand")
    return recording


In [62]:
motion_recording_service_1 = motion_recording["Segment Position"].loc[7650:7800]
motion_recording_service_2 = motion_recording["Segment Position"].loc[9450:9600]

frames_number = 150

fig = plt.figure()
ax = fig.add_subplot(projection="3d")

ax.set(xlim3d=(-1, 1), xlabel='X')
ax.set(ylim3d=(-1, 1), ylabel='Y')
ax.set(zlim3d=(-1, 1), zlabel='Z')

skeleton_0 = [ax.plot([], [], [], c="r")[0] for _ in SKELETON_CONNECTION_MAP]
skeleton_1 = [ax.plot([], [], [], c="b")[0] for _ in SKELETON_CONNECTION_MAP]
skeleton_2 = [ax.plot([], [], [], c="g")[0] for _ in SKELETON_CONNECTION_MAP]

motion_recording_service_1_0 = motion_recording_service_1.copy()
motion_recording_service_1_centred_0 = centre(motion_recording_service_1).copy()
motion_recording_service_1_centred_confronted = confront(motion_recording_service_1)

ani_0 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True, fargs=(motion_recording_service_1_0,skeleton_0))
ani_1 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True, fargs=(motion_recording_service_1_centred_0,skeleton_1))
ani_2 = animation.FuncAnimation(fig, update, frames=frames_number, interval=40, repeat=True, fargs=(motion_recording_service_1_centred_confronted,skeleton_2))

is_paused = False
def toggle_pause(anis):
    global is_paused
    if is_paused:
        for ani in anis:
            ani.resume()
    else:
        for ani in anis:
            ani.pause()
    is_paused = not is_paused


fig.canvas.mpl_connect('button_press_event', lambda event: toggle_pause([ani_0, ani_1, ani_2]))

plt.show()

C:\Users\gaoch\AppData\Local\Temp\ipykernel_11208\302772534.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recording[columns_with_x] = recording[columns_with_x].sub(recording["Pelvis x"],axis='index')
C:\Users\gaoch\AppData\Local\Temp\ipykernel_11208\302772534.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recording[columns_with_y] = recording[columns_with_y].sub(recording["Pelvis y"],axis='index')
C:\Users\gaoch\AppData\Local\Temp\ipykernel_11208\302772534.py:19: SettingWithCopyWarning: 
A valu

7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7661
7662
7663
7664
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7687
7688
7689
7690
7691
7692
7693
7694
7695
7696
7697
7698
7699
7700
7701
7702
7703
7704
7705
7706
7707
7708
7709
7710
7711
7712
7713
7714
7715
7716
7717
7718
7719
7720
7721
7722
7723
7724
7725
7726
7727
7728
7729
7730
7731
7732
7733
7734
7735
7736
7737
7738
7739
7740
7741
7742
7743
7744
7745
7746
7747
7748
7749
7750
7751
7752
7753
7754
7755
7756
7757
7758
7759
7760
7761
7762
7763
7764
7765
7766
7767
7768
7769
7770
7771
7772
7773
7774
7775
7776
7777
7778
7779
7780
7781
7782
7783
7784
7785
7786
7787
7788
7789
7790
7791
7792
7793
7794
7795
7796
7797
7798
7799
7800


In [40]:
print(sin_cos["sin"][7650])

0.8859687371663989
